In [1]:
import os

MAX_BATCHES = 113

def checkpoint_time(checkpoint: str):
    checkpoint = checkpoint.split('-')
    epoch = int(checkpoint[0][5:])
    batch = int(checkpoint[1][5:-3])
    return epoch, batch

def make_schedule(n_batches):
    n = int((n_batches + 1) / 2)
    ret = [0, n]
    n = int((n + 1) / 2)
    i = 1
    end = False
    while True:
        prev = ret[-i:]
        for p in prev:
            if p-n not in ret and p-n > 0 : ret.append(p - n)
            if p+n not in ret and p+n < n_batches: ret.append(p + n)

        if end or n == 0: break
        end = n == 1
        n = int((n+1) / 2)
        i = i * 2
    return list(ret)

def normalize_batch(epoch, batch):
    return epoch + (batch / MAX_BATCHES)

def get_checkpoints(run_name: str):
    weights_dir = os.path.join('runs', 'detect', run_name, 'weights')
    checkpoints = sorted(
        [c for c in os.listdir(weights_dir) if 'batch' in c],
        key=lambda x: normalize_batch(*checkpoint_time(x))
    )
    return weights_dir, checkpoints

In [2]:
from ultralytics import YOLO
from custom_eval import eval_video, get_test_files, BINARY_KEYS
import json
TEST_DIR = "datasets/04_Test_Datasets/"

test_files = get_test_files(TEST_DIR)

def eval_checkpoints(run_name, frame_conf, class_conf, max_epochs=None):
    weights_dir, checkpoints = get_checkpoints(run_name)
    n_epochs = len(set([checkpoint_time(c)[0] for c in checkpoints]))
    n_batches = len(set([checkpoint_time(c)[1] for c in checkpoints]))
    checkpoints_2d = [
        [c for c in checkpoints if checkpoint_time(c)[0] == e]
        for e in range(n_epochs)
    ]
    schedule = make_schedule(n_batches)
    print(schedule)
    for batch in schedule:
        for epoch in range(min(n_epochs, max_epochs)):
            checkpoint = checkpoints_2d[epoch][batch]
            print(f"EVAL ON EPOCH {epoch}, BATCH {batch} ({checkpoint})")
            model = YOLO(os.path.join(weights_dir, checkpoint))
            res = eval_video(model, test_files, frame_conf, class_conf, BINARY_KEYS)
            real_batch = checkpoint_time(checkpoint)[1]
            res['epoch'] = epoch
            res['batch'] = real_batch
            with open(os.path.join('runs', 'detect', run_name, 'bias.jsonl'), 'a') as f:
                f.write(json.dumps(res) + "\n")



In [3]:
eval_checkpoints('binary_ft_checkpoints', frame_conf=1e-2, class_conf=0.5, max_epochs=5)

[0, 7, 3, 11, 1, 5, 9, 10, 12, 2, 4, 6, 8]
EVAL ON EPOCH 0, BATCH 0 (epoch0-batch0.pt)
starting eval...


  0%|          | 0/81 [00:00<?, ?it/s]

<class 'ultralytics.models.yolo.detect.predict.DetectionPredictor'>


/home/coinse/Documents/lm/spring2024/CS570-Video-Classification/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv2d(input, weight, bias, self.stride,
/home/coinse/Documents/lm/spring2024/CS570-Video-Classification/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 81/81 [03:02<00:00,  2.25s/it]


NUM CORRECT: 22 / 81 (0.27)
LABELS DISTRIBUTION:
{'None': 12, 'bird': 8, 'plane': 61}
PREDICTIONS DISTRIBUTION:
{'None': {'False_Neg': 10,
          'False_Pos': 6,
          'True_Neg': 63,
          'True_Pos': 2,
          'guess_rate': 0.09876543209876543},
 'bird': {'False_Neg': 6,
          'False_Pos': 47,
          'True_Neg': 26,
          'True_Pos': 2,
          'guess_rate': 0.6049382716049383},
 'plane': {'False_Neg': 43,
           'False_Pos': 6,
           'True_Neg': 14,
           'True_Pos': 18,
           'guess_rate': 0.2962962962962963}}
EVAL ON EPOCH 1, BATCH 0 (epoch1-batch0.pt)
starting eval...


...

NUM CORRECT: 52 / 81 (0.64)
LABELS DISTRIBUTION:
{'None': 12, 'bird': 8, 'plane': 61}
PREDICTIONS DISTRIBUTION:
{'None': {'False_Neg': 11,
          'False_Pos': 1,
          'True_Neg': 68,
          'True_Pos': 1,
          'guess_rate': 0.024691358024691357},
 'bird': {'False_Neg': 2,
          'False_Pos': 23,
          'True_Neg': 50,
          'True_Pos': 6,
          'guess_rate': 0.35802469135802467},
 'plane': {'False_Neg': 16,
           'False_Pos': 5,
           'True_Neg': 15,
           'True_Pos': 45,
           'guess_rate': 0.6172839506172839}}
EVAL ON EPOCH 4, BATCH 8 (epoch4-batch80.pt)
starting eval...


  0%|          | 0/81 [00:00<?, ?it/s]

<class 'ultralytics.models.yolo.detect.predict.DetectionPredictor'>


100%|██████████| 81/81 [02:57<00:00,  2.19s/it]

NUM CORRECT: 54 / 81 (0.67)
LABELS DISTRIBUTION:
{'None': 12, 'bird': 8, 'plane': 61}
PREDICTIONS DISTRIBUTION:
{'None': {'False_Neg': 10,
          'False_Pos': 2,
          'True_Neg': 67,
          'True_Pos': 2,
          'guess_rate': 0.04938271604938271},
 'bird': {'False_Neg': 2,
          'False_Pos': 21,
          'True_Neg': 52,
          'True_Pos': 6,
          'guess_rate': 0.3333333333333333},
 'plane': {'False_Neg': 15,
           'False_Pos': 4,
           'True_Neg': 16,
           'True_Pos': 46,
           'guess_rate': 0.6172839506172839}}


In [4]:
eval_checkpoints('full_ft_checkpoints', frame_conf=1e-2, class_conf=0.5, max_epochs=5)


[0, 7, 3, 11, 1, 5, 9, 10, 12, 2, 4, 6, 8]
EVAL ON EPOCH 0, BATCH 0 (epoch0-batch0.pt)
starting eval...


  0%|          | 0/81 [00:00<?, ?it/s]

<class 'ultralytics.models.yolo.detect.predict.DetectionPredictor'>


100%|██████████| 81/81 [04:07<00:00,  3.05s/it]


NUM CORRECT: 8 / 81 (0.10)
LABELS DISTRIBUTION:
{'None': 12, 'bird': 8, 'plane': 61}
PREDICTIONS DISTRIBUTION:
{'bird': {'False_Neg': 0,
          'False_Pos': 73,
          'True_Neg': 0,
          'True_Pos': 8,
          'guess_rate': 1.0}}
EVAL ON EPOCH 1, BATCH 0 (epoch1-batch0.pt)
starting eval...


...

NUM CORRECT: 26 / 81 (0.32)
LABELS DISTRIBUTION:
{'None': 12, 'bird': 8, 'plane': 61}
PREDICTIONS DISTRIBUTION:
{'bird': {'False_Neg': 1,
          'False_Pos': 53,
          'True_Neg': 20,
          'True_Pos': 7,
          'guess_rate': 0.7407407407407407},
 'plane': {'False_Neg': 42,
           'False_Pos': 2,
           'True_Neg': 18,
           'True_Pos': 19,
           'guess_rate': 0.25925925925925924}}
EVAL ON EPOCH 4, BATCH 8 (epoch4-batch80.pt)
starting eval...


  0%|          | 0/81 [00:00<?, ?it/s]

<class 'ultralytics.models.yolo.detect.predict.DetectionPredictor'>


100%|██████████| 81/81 [03:01<00:00,  2.24s/it]

NUM CORRECT: 26 / 81 (0.32)
LABELS DISTRIBUTION:
{'None': 12, 'bird': 8, 'plane': 61}
PREDICTIONS DISTRIBUTION:
{'None': {'False_Neg': 9,
          'False_Pos': 0,
          'True_Neg': 69,
          'True_Pos': 3,
          'guess_rate': 0.037037037037037035},
 'bird': {'False_Neg': 1,
          'False_Pos': 53,
          'True_Neg': 20,
          'True_Pos': 7,
          'guess_rate': 0.7407407407407407},
 'plane': {'False_Neg': 45,
           'False_Pos': 2,
           'True_Neg': 18,
           'True_Pos': 16,
           'guess_rate': 0.2222222222222222}}
